## Mount Google Drive in Colab

In [0]:
from google.colab import drive
drive.mount('/content/drive')

## Perform imports

In [0]:
import numpy as np
%tensorflow_version 1.x
import keras
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import SGD
from keras.applications.vgg16 import VGG16, preprocess_input as ppi_vgg16
import json

TensorFlow 1.x selected.


Using TensorFlow backend.


## Load balanced dataset

In [0]:
data = np.load('/content/drive/My Drive/Final Capstone/Balanced Data.npy')

## Separate image data from label data

In [0]:
# image data
X = data[:, :-1]
# label data
Y = data[:, -1:]
del data

## Reshape image and label data and create one-hot encoding for label data

In [0]:
# reshape image data to 50 x 50 x 3 format
X = np.reshape(X, (X.shape[0], 50, 50, 3), order='C')
# reshape label data to 1-D array
Y = np.reshape(Y, (Y.shape[0]))
# create one-hot encoding for label data
Y = keras.utils.to_categorical(Y, 2, 'float32')

## VGG16 Models

Four models are created using using the VGG16 model as a basis and fit to the training set.  Two models are transfer learning models based on freezing the weights from the 'imagenet' training.  In one of these models, only a dense prediction layer is added.  In the other, a hidden dense layer and a dense prediction layer are added.  Two other models are created that simply make use of the architecture of the VGG16 model.  These models do not use the weights from 'imagenet' training.  Similarly to the models that use the pretrained weights, one of these models only has a dense prediction layer while the other has a hidden dense layer and a dense prediction layer.  The same method that was used to train the other 2-D Convolutional Neural Networks is used to train these models.

In [0]:
# fraction of data to use for validation--taken from the bottom of the dataset by default in Keras
val_split = 0.2
# initial learning rate used in optimizer
initial_lr = 0.01
# width of hidden layer used prior to prediction dense layer
hidden_dense_size = 64

In [0]:
# see 'Run NN Architectures -- #4.ipynb' for commented version of this function
def optimize_model(model, X, Y, val_split, initial_lr):
  epoch_count = 0
  min_epochs = 5
  max_epochs = 20
  history_dict = None
  sgd = SGD(lr=initial_lr)
  sgd_lr = sgd.get_config()['lr']
  # while loop based on model train accuracy convergence
  while (True):
    # while loop based on learning rate reduction
    while (True):
      model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
      old_weights = model.get_weights()
      history = model.fit(X, Y, epochs=1, validation_split=val_split, verbose=1)

      if (history_dict == None):
        history_dict = history.history
        break
      else:
        last_loss = history_dict['loss'][len(history_dict['loss']) - 1]
        curr_loss = history.history['loss'][0]
        if (curr_loss <= last_loss):
          for key in history_dict.keys():
            history_dict[key].append(history.history[key][0])
          break
        else:
          sgd = SGD(lr=sgd_lr/2)
          sgd_lr = sgd.get_config()['lr']
          model.set_weights(old_weights)

    epoch_count += 1

    if (epoch_count >= min_epochs):
      if (epoch_count == max_epochs):
        break
      else:
        if (history_dict['val_acc'][epoch_count - 1] < history_dict['val_acc'][epoch_count - 2]):
          break
  return history_dict

In [0]:
# preprocess input so that it is compatible with how input was preprocessed for training of VGG16 model
X_vgg16 = ppi_vgg16(X * 255)
del X

In [0]:
# VGG16 model with frozen pretrained weights and prediction dense layer

model_vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=X_vgg16.shape[1:4], pooling='max')

# freeze pretrained weights
for layer in model_vgg16.layers:
  layer.trainable = False

input_layer = Input(shape=X_vgg16.shape[1:4])
model_vgg16_layer = model_vgg16(input_layer)
output_layer = Dense(units=2, activation='softmax')(model_vgg16_layer)
model = Model(input_layer, output_layer)

output_dict = optimize_model(model, X_vgg16, Y, val_split, initial_lr)

with open('/content/drive/My Drive/Final Capstone/Model Logs/vgg16_d.json', 'w') as f:
  json.dump(output_dict, f)

In [0]:
# VGG16 model with frozen pretrained weights, hidden dense layer, and dense prediction layer

model_vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=X_vgg16.shape[1:4], pooling='max')

# freeze pretrained weights
for layer in model_vgg16.layers:
  layer.trainable = False

input_layer = Input(shape=X_vgg16.shape[1:4])
model_vgg16_layer = model_vgg16(input_layer)
output_layer = Dense(units=hidden_dense_size, activation='relu')(model_vgg16_layer)
output_layer1 = Dense(units=2, activation='softmax')(output_layer)
model = Model(input_layer, output_layer1)

output_dict = optimize_model(model, X_vgg16, Y, val_split, initial_lr)

with open('/content/drive/My Drive/Final Capstone/Model Logs/vgg16_d_d.json', 'w') as f:
  json.dump(output_dict, f)

In [0]:
# restore data to form from before preprocessing for VGG16
del X_vgg16
data = np.load('/content/drive/My Drive/Final Capstone/Balanced Data.npy')
X = data[:, :-1]
Y = data[:, -1:]
del data
X = np.reshape(X, (X.shape[0], 50, 50, 3), order='C')
Y = np.reshape(Y, (Y.shape[0]))
Y = keras.utils.to_categorical(Y, 2, 'float32')

In [0]:
# VGG16 model without pretrained weights and with dense prediction layer

model_vgg16 = VGG16(include_top=False, weights=None, input_shape=X.shape[1:4], pooling='max')
input_layer = Input(shape=X.shape[1:4])
model_vgg16_layer = model_vgg16(input_layer)
output_layer = Dense(units=2, activation='softmax')(model_vgg16_layer)
model = Model(input_layer, output_layer)

output_dict = optimize_model(model, X, Y, val_split, initial_lr)

with open('/content/drive/My Drive/Final Capstone/Model Logs/vgg16_d_np.json', 'w') as f:
  json.dump(output_dict, f)

In [0]:
# VGG16 model without pretrained weights and with hidden dense layer and dense prediction layer

model_vgg16 = VGG16(include_top=False, weights=None, input_shape=X.shape[1:4], pooling='max')
input_layer = Input(shape=X.shape[1:4])
model_vgg16_layer = model_vgg16(input_layer)
output_layer = Dense(units=hidden_dense_size, activation='relu')(model_vgg16_layer)
output_layer1 = Dense(units=2, activation='softmax')(output_layer)
model = Model(input_layer, output_layer1)

output_dict = optimize_model(model, X, Y, val_split, initial_lr)

with open('/content/drive/My Drive/Final Capstone/Model Logs/vgg16_d_d_np.json', 'w') as f:
  json.dump(output_dict, f)